In [ ]:
import numpy as np
import pandas as pd
import pickle5 as pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy

In [ ]:
price = np.array([0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.1199,
                       0.1199, 0.1199, 0.1199, 0.1199, 0.1199, 0.2499, 0.2499, 0.2499, 0.1199, 0.1199, 0.1199,
                       0.1199, 0.1199, 0.1199])
step_1h = 0
gas_price = np.array([0.15820997, 0.17198271, 0.14196519, 0.13808057, 0.1345491, 0.13348966,
                      0.13313651, 0.12995819, 0.13278337, 0.14267149, 0.15962256, 0.19352469])

In [ ]:
tou_price = price[step_1h]  # electricity buy price from the grid
fit_price = 0.04  # electricity sell price
p_ng = self.gas_price[4]  # natural gas price (May)
p_h2 = 5  # hydrogen gas price ($/kg)

# s_bg = 1.2  # price ratio of buying from grid to p2p
# s_sg = 0.8  # price ratio of selling from grid to p2p

k_we = 0.8  # water electrolyser efficiency
k_fc_e = 0.3  # fuel cell to electricity efficiency
k_fc_h = 0.55  # fuel cell to heat efficiency
k_gb = 0.9  # gas boiler efficiency
k_ng2q = 8.816  # natural gas(m3) to Q(KWh) ratio
k_h2q = 33.33  # hydrogen(kg) to Q(KWh) ratio

c_p = 3 * tou_price  # electricity penalty coefficient
c_h = 3 * p_ng  # heat penalty coefficient

beta_gas = 0.245  # carbon intensity kg/kwh
beta_elec = 0.683  # carbon intensity kg/kwh

reward = 0
E_dif = 0  # electricity balance difference (Kwh)
h2_dif = 0  # h2 balance difference (kg)
H_dif = 0  # heat balance difference (m3)

# ----------------------------- DA market ------------------------------- #

# electricity orderbook
E_da_quantity = action[1] * 100  # kwh
H_da_quantity = action[3] * 100  # kwh
res_da_elec = {
    "name": "res",
    "da_price": action[0], "tou_price": tou_price, "fit_price": fit_price,
    "da_quantity": E_da_quantity, "reward": 0}
com_da_elec = {
    "name": "com", "da_price": self.other_mg_da_info.iloc[self.step_1h, 0],
    "da_quantity": self.other_mg_da_info.iloc[self.step_1h, 1], "reward": 0}
ind_da_elec = {
    "name": "ind", "da_price": self.other_mg_da_info.iloc[self.step_1h, 4],
    "da_quantity": self.other_mg_da_info.iloc[self.step_1h, 5], "reward": 0}

# heat orderbook
tou_price_h = self.gas_price[4] / (k_ng2q * k_gb)  # equivalent heat price $/kwh (heat provided by gas boiler)
fit_price_h = 0  # can not sell heat back to the external network
res_da_heat = {
    "name": "res",
    "da_price": action[2], "tou_price": tou_price_h, "fit_price": fit_price_h,
    "da_quantity": H_da_quantity, "reward": 0}
com_da_heat = {
    "name": "com", "da_price": self.other_mg_da_info.iloc[self.step_1h, 2],
    "da_quantity": self.other_mg_da_info.iloc[self.step_1h, 3], "reward": 0}
ind_da_heat = {
    "name": "ind", "da_price": self.other_mg_da_info.iloc[self.step_1h, 6],
    "da_quantity": self.other_mg_da_info.iloc[self.step_1h, 7], "reward": 0}


In [ ]:
alpha_co2 = 0.0316  # carbon tax $/kg
tou_price = res_da_info["tou_price"]
fit_price = res_da_info["fit_price"]

# change price from (-1,1) to real price
res_da_info["da_price"] = (res_da_info["da_price"] + 1) * 0.5 * (tou_price - fit_price) + fit_price
com_da_info["da_price"] = (com_da_info["da_price"] + 1) * 0.5 * (tou_price - fit_price) + fit_price
ind_da_info["da_price"] = (ind_da_info["da_price"] + 1) * 0.5 * (tou_price - fit_price) + fit_price

In [ ]:
res_data, com_data, ind_data = deepcopy(res_da_info), deepcopy(com_da_info), deepcopy(ind_da_info)
for da_info in [res_data, com_data, ind_data]:
    if da_info["da_quantity"] < 0:
        da_info["role"] = "seller"
        da_info["da_quantity"] = -da_info["da_quantity"]
    else:
        da_info["role"] = "buyer"

df = pd.DataFrame(data=[res_data, com_data, ind_data])
buy_order_book = df[df.role == "buyer"].sort_values(by='da_price', ascending=False)  # descending price
sell_order_book = df[df.role == "seller"].sort_values(by='da_price')  # ascending price